In [2]:
import pandas as pd

# S&P 500 Prices


In [3]:
df_SP = pd.read_csv("./Raw/SPX.csv")

#number of null values
sum(df_SP['Volume'] == 0)

#number of rows
len(df_SP.index)


23323

# Bitcoin Sentiment 

In [ ]:
df_BS = pd.read_csv("./Raw/BitcoinSent.csv")

#removed non-date data in the 'Date' header
df_BS['Date'] = pd.to_datetime(df_BS['Date'], errors='coerce')
df_BS = df_BS[df_BS['Date'].notna()]

#sort in ascending date
df_BS = df_BS.sort_values(by='Date', ascending=True)
df_BS = df_BS.reset_index(drop=True)

#testing
df_BS.tail



<bound method NDFrame.tail of                Date                                               text  \
0        2014-09-18  LIVE: Profit = $414.71 (1.31 %). BUY B75.77 @ ...   
1        2014-09-18  Current price: 418.77$ $BTCUSD $btc #bitcoin 2...   
2        2014-09-18  1 #BTC (#Bitcoin) quotes:\n$423.60/$424.80 #Bi...   
3        2014-09-18  In the last 10 mins, there were arb opps spann...   
4        2014-09-18  Be judicious, buy your Bitcoins at https://Bit...   
...             ...                                                ...   
19344014 2019-11-23  €400 million investment in Blockchain and AI t...   
19344015 2019-11-23  BTC/USD | $BTCUSD | $BTC $USD\n\nBitcoin Outlo...   
19344016 2019-11-23  BTC\n\n長期的目線\n\n現在のトライアングル収束までに要した期間と人々の関心から、\...   
19344017 2019-11-23  SPECIAL DEAL TO ANYONE HAS CASH APP OR BITCOIN...   
19344018 2019-11-23  $BTC - an update on the longer term view for B...   

         Sentiment  
0         Negative  
1         Negative  
2         Negative